
# 05 — Interactive Visual Analytics with Folium

This notebook builds the **interactive maps** required by the rubric and saves each as an HTML artifact you can open and screenshot for the slides.

**Inputs**
- `data/launches_clean.csv`

**Artifacts (saved in `./artifacts`)**
- `(Map 1) Global Launch Sites Map.html`
- `(Map 2) Launch Outcomes Map.html`
- `(Map 3) Proximity Map — <SITE>.html`


## Setup

In [26]:

# If needed:
# !pip install pandas folium

import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster, MousePosition, MeasureControl
from pathlib import Path

DATA_DIR = Path("./data")
ARTIFACTS_DIR = Path("./artifacts")
ARTIFACTS_DIR.mkdir(exist_ok=True, parents=True)

CSV_PATH = DATA_DIR / "launches_clean.csv"
assert CSV_PATH.exists(), "Missing data/launches_clean.csv — run previous notebooks."

df = pd.read_csv(CSV_PATH, parse_dates=["date_utc"], dtype={"flight_number":"Int64","year":"Int64"})
df['site_lat'] = pd.to_numeric(df['site_lat'], errors='coerce')
df['site_lon'] = pd.to_numeric(df['site_lon'], errors='coerce')
df['launch_success'] = df['launch_success'].map({True:1, False:0, "True":1, "False":0, 1:1, 0:0}).fillna(0).astype(int)

# Helper for success labels/colors
def outcome_color(row):
    txt = str(row.get('landing_outcome', '')).lower()
    if 'success' in txt and 'drone' in txt:
        return 'green'
    if 'success' in txt and ('ground' in txt or 'rtls' in txt):
        return 'blue'
    if 'success' in txt:
        return 'green'
    if 'failure' in txt and 'drone' in txt:
        return 'red'
    if 'failure' in txt and ('ground' in txt or 'rtls' in txt):
        return 'orange'
    if 'no attempt' in txt:
        return 'gray'
    return 'lightgray'

def popup_html(row):
    date_val = row.get('date_utc')
    if isinstance(date_val, pd.Timestamp):
        date_str = date_val.strftime('%Y-%m-%d')
    elif pd.isna(date_val):
        date_str = '—'
    else:
        # if CSV loaded it as string
        date_str = str(date_val)[:10]

    mass = row.get('payload_mass_kg')
    mass_str = f"{mass:.0f}" if pd.notna(mass) else '—'

    success_str = 'Yes' if int(row.get('launch_success', 0)) == 1 else 'No'

    return folium.Popup(
        f"""<b>Flight #{int(row['flight_number']) if pd.notna(row['flight_number']) else '—'}</b><br>
        Date (UTC): {date_str}<br>
        Site: {row.get('launch_site','—')}<br>
        Payload: {row.get('payload_name','—')} ({mass_str} kg)<br>
        Orbit: {row.get('orbit','—')}<br>
        Landing: {row.get('landing_outcome','—')}<br>
        Success: {success_str}""",
        max_width=350
    )


# Launch-level aggregation for site markers
site_stats = (df.groupby(['launch_site','site_lat','site_lon'], dropna=True)
                .agg(n_launches=('flight_number','nunique'),
                     success_rate=('launch_success', 'mean'))
                .reset_index())
site_stats['success_rate_pct'] = (site_stats['success_rate']*100).round(1)
site_stats.head(3)


,launch_site,site_lat,site_lon,n_launches,success_rate,success_rate_pct
0,CCSFS SLC 40,28.561857,-80.577366,99,0.981132,98.1
1,KSC LC 39A,28.608058,-80.603956,52,1.000000,100.0
2,VAFB SLC 4E,34.632093,-120.610829,28,0.966667,96.7


## Global Launch Sites Map

In [29]:

# Center roughly on the geographic mean of sites
center_lat = site_stats['site_lat'].mean()
center_lon = site_stats['site_lon'].mean()
m1 = folium.Map(location=[center_lat, center_lon], zoom_start=2, tiles='OpenStreetMap')

mc = MarkerCluster().add_to(m1)

for _, r in site_stats.iterrows():
    folium.Marker(
        location=[r['site_lat'], r['site_lon']],
        popup=folium.Popup(f"""<b>{r['launch_site']}</b><br>
            Launches: {int(r['n_launches'])}<br>
            Success rate: {r['success_rate_pct']}%""", max_width=280),
        tooltip=r['launch_site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(mc)

# Add helpful plugins
MousePosition(position='topright', prefix='Lat/Lon:').add_to(m1)
m1.add_child(MeasureControl())

title_html = folium.Element('<h4 style="position: fixed; top: 10px; left: 50px; z-index: 9999; background: rgba(255,255,255,.8); padding:6px 10px; border-radius:6px">(Map 1) Global Launch Sites — markers show total launches & success rate</h4>')
m1.get_root().html.add_child(title_html)

out1 = ARTIFACTS_DIR / "(Map 1) Global Launch Sites Map.html"
m1.save(str(out1))
m1



**Explanation (template):**  
All SpaceX Falcon 9 launch sites are marked. Popups summarize total launches and success rates, showing heavy usage at Cape Canaveral and Kennedy LC-39A, with Vandenberg/Starbase as secondary sites.


## Launch Outcomes Map

In [33]:

# Use payload-level rows; add slight jitter so overlapping launches at same site are distinguishable
rng = np.random.default_rng(42)
df_points = df.dropna(subset=['site_lat','site_lon']).copy()
df_points['lat_j'] = df_points['site_lat'] + rng.normal(0, 0.02, size=len(df_points))  # ~2km jitter
df_points['lon_j'] = df_points['site_lon'] + rng.normal(0, 0.02, size=len(df_points))

m2 = folium.Map(location=[center_lat, center_lon], zoom_start=2, tiles='OpenStreetMap')
mc2 = MarkerCluster().add_to(m2)

for _, r in df_points.iterrows():
    folium.CircleMarker(
        location=[r['lat_j'], r['lon_j']],
        radius=5,
        color=outcome_color(r),
        fill=True, fill_opacity=0.7,
        popup=popup_html(r)
    ).add_to(mc2)

MousePosition(position='topright', prefix='Lat/Lon:').add_to(m2)
m2.add_child(MeasureControl())

title_html = folium.Element('<h4 style="position: fixed; top: 10px; left: 50px; z-index: 9999; background: rgba(255,255,255,.8); padding:6px 10px; border-radius:6px">(Map 2) Launch Outcomes — color shows landing outcome (success/failure, drone/ground/no attempt)</h4>')
m2.get_root().html.add_child(title_html)

out2 = ARTIFACTS_DIR / "(Map 2) Launch Outcomes Map.html"
m2.save(str(out2))
m2



**Explanation (template):**  
Markers are color-coded by landing outcome: **green/blue** for successes (drone/ground), **red/orange** for failures, **gray** for no-attempt/unknown. This view shows most outcomes as successful, especially in recent years.


## Launch Site Proximity Map


We select a single launch site and visualize distances to nearby **coastline**, **highway**, and **railway** points of interest (POIs).

> **Note:** In the original lab this is often done interactively by clicking the map to collect POIs.  
> To keep this notebook reproducible offline, we include a small dictionary of representative POIs for common SpaceX sites. You can change the `SELECTED_SITE` and/or the POI coordinates to your own.


In [38]:

import math

# Haversine great-circle distance (km)
def haversine(coord1, coord2):
    R = 6371.0
    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

# Known launch site names in SpaceX API:
# 'KSC LC 39A', 'CCSFS SLC 40', 'VAFB SLC 4E', 'STLS' (Starbase)
# Provide approximate POIs (manually curated; adjust if needed)
POI_BY_SITE = {
    "KSC LC 39A": {
        "site": (28.60839, -80.60433),
        "coastline": (28.638, -80.57),
        "highway": (28.539, -80.669),   # NASA Causeway vicinity
        "railway": (28.5729, -80.678),  # KSC rail spur (approx)
    },
    "CCSFS SLC 40": {
        "site": (28.5618571, -80.577366),
        "coastline": (28.57, -80.56),
        "highway": (28.563, -80.585),
        "railway": (28.562, -80.59),
    },
    "VAFB SLC 4E": {
        "site": (34.632093, -120.610829),
        "coastline": (34.64, -120.66),
        "highway": (34.589, -120.617),  # Hwy 1
        "railway": (34.636, -120.607),  # Union Pacific (approx)
    },
    "STLS": {  # Starbase
        "site": (25.9972, -97.1566),
        "coastline": (25.996, -97.157),
        "highway": (25.9984, -97.1779), # Boca Chica Blvd (TX 4)
        "railway": (25.964, -97.155),
    },
}

# Choose a site that exists in our data; default to the most frequent
site_counts = df.groupby('launch_site')['flight_number'].nunique().sort_values(ascending=False)
DEFAULT_SITE = next((s for s in site_counts.index if s in POI_BY_SITE), site_counts.index[0])

SELECTED_SITE = DEFAULT_SITE  # <- change if desired

coords = POI_BY_SITE.get(SELECTED_SITE)
if not coords:
    raise ValueError(f"No POIs configured for site '{SELECTED_SITE}'. Please update POI_BY_SITE.")

site_coord = coords['site']
coast_coord = coords['coastline']
highway_coord = coords['highway']
rail_coord = coords['railway']

d_coast = haversine(site_coord, coast_coord)
d_highway = haversine(site_coord, highway_coord)
d_rail = haversine(site_coord, rail_coord)

m3 = folium.Map(location=list(site_coord), zoom_start=12, tiles='OpenStreetMap')

# Site marker
folium.Marker(site_coord, tooltip=f"{SELECTED_SITE} (Launch Site)",
              icon=folium.Icon(color='blue', icon='rocket', prefix='fa')).add_to(m3)

# POI markers
folium.Marker(coast_coord, tooltip=f"Coastline (~{d_coast:.2f} km)",
              icon=folium.Icon(color='green')).add_to(m3)
folium.Marker(highway_coord, tooltip=f"Highway (~{d_highway:.2f} km)",
              icon=folium.Icon(color='purple')).add_to(m3)
folium.Marker(rail_coord, tooltip=f"Railway (~{d_rail:.2f} km)",
              icon=folium.Icon(color='red')).add_to(m3)

# Draw lines & labels
for name, coord, dist in [
    ("Coastline", coast_coord, d_coast),
    ("Highway", highway_coord, d_highway),
    ("Railway", rail_coord, d_rail),
]:
    folium.PolyLine([site_coord, coord], color='gray', weight=2, opacity=0.8).add_to(m3)
    folium.map.Marker(
        coord,
        icon=folium.DivIcon(html=f'<div style="font-size: 12px; color: #111; background: rgba(255,255,255,.8); padding:2px 4px; border-radius:4px;">{name}: {dist:.2f} km</div>')
    ).add_to(m3)

MousePosition(position='topright', prefix='Lat/Lon:').add_to(m3)
m3.add_child(MeasureControl())

title_html = folium.Element(f'<h4 style="position: fixed; top: 10px; left: 50px; z-index: 9999; background: rgba(255,255,255,.8); padding:6px 10px; border-radius:6px">(Map 3) Proximity — {SELECTED_SITE}</h4>')
m3.get_root().html.add_child(title_html)

out3 = ARTIFACTS_DIR / f"(Map 3) Proximity Map — {SELECTED_SITE}.html"
m3.save(str(out3))

print("Selected site:", SELECTED_SITE)
print(f"Distances (km) — Coastline: {d_coast:.2f}, Highway: {d_highway:.2f}, Railway: {d_rail:.2f}")
m3


Selected site: CCSFS SLC 40
Distances (km) — Coastline: 1.92, Highway: 0.76, Railway: 1.23



**Explanation (template):**  
The proximity map shows the selected launch pad and approximate distances to the nearest coastline, highway, and railway. Short distances to the coast and main access roads are consistent with range safety and logistics constraints.
